# Intra-Annual Sea Level Rankings

In this notebook we'll explore sea level rankings on an intra-annual scale.

## Setup

As with previous sections, we first need to import the necessary libraries, establish our input/output directories, and set up some basic plotting rules. We'll do this by running another notebook called [setup](notebooks/setup.ipynb) and import our plotting functions, which will also set our data and output paths. If you have not run the [datawrangling notebook](notebooks/SL_Data_Wrangling.ipynb), you will need to do this before running this notebook. Note that this notebook is also largely a repeat of the [anomaly](notebooks/regional_and_local/SL_anomaly_annual.ipynb) notebook. 

In [ ]:
%run ../setup.ipynb

import plotly.io as pio
import calendar

# check to make sure that data_dir/rsl_daily_hawaii.nc exists, if not, make warning to run datawrangling notebook
if not (data_dir / 'rsl_hawaii.nc').exists():
    print('rsl_hawaii.nc not found in '+ str(data_dir) +  '. Please run the data wrangling notebook first')
else:
    print('rsl_hawaii.nc found in '+ str(data_dir) +  '. Proceed.')

Import the sea level data that we've already processed.

In [ ]:
#import rsl_daily
rsl_hourly = xr.open_dataset(data_dir/ 'rsl_hawaii_noaa.nc')
rsl_hourly['sea_level_mhhw'] = rsl_hourly['sea_level'] - rsl_hourly['MHHW']

In [ ]:
# make dict of station ID to station name
station_dict = {stid: rsl_hourly['station_name'].sel(station_id=stid).values.item() for stid in rsl_hourly['station_id'].values}

# output this prettily
for stid, name in station_dict.items():
    print(f'Station ID: {stid}, Name: {name}')

In [ ]:
#choose a station
station_id = '1612340'

Define a function to retrieve the top (or bottom) 10 events by month and by station, using the sea level (m, MHHW) variable we just created in rsl_hourly

In [ ]:
def get_top_ten(rsl_subset, station_id, month, mode='max'):
    # Convert data to a pandas Series
    sea_level_series = rsl_subset.sea_level_mhhw.sel(station_id=station_id).where(rsl_hourly['time.month'] == month, drop=True).to_series()

   # Select top 100 values based on the mode
    if mode == 'max':
        top_values = sea_level_series.nlargest(100)
    elif mode == 'min':
        top_values = sea_level_series.nsmallest(100)
    else:
        raise ValueError('mode must be either "max" or "min"')

    # Filter to find unique events spaced by at least 3 days
    filtered_dates = []
    top_10_values = pd.Series()

    for date, value in top_values.items():
        if all(abs((date - pd.to_datetime(added_date)).days) > 3 for added_date in filtered_dates):
            filtered_dates.append(date)
            top_10_values[date] = value
        if len(filtered_dates) == 10:
            break
    rank = np.arange(1,11)

    station_name = str(rsl_subset['station_name'].sel(station_id=str(station_id)).values)

    top_10_values = pd.DataFrame({'rank in month': rank, 'date': top_10_values.index, 'month': month, 'sea level (m, MHHW)': top_10_values.values})  
    top_10_values['station_name'] = station_name
    top_10_values['station_id'] = station_id
    top_10_values['type'] = mode

    #round the date to the nearest hour
    top_10_values['date'] = top_10_values['date'].dt.round('h')

    return top_10_values

In [ ]:
# get the top ten highest hourly sea level values for each station
top_ten_values = pd.DataFrame()
for month in range(1,13):
    top_ten_values = pd.concat([top_ten_values, get_top_ten(rsl_hourly, station_id=station_id, month=month, mode='max')])

In [ ]:
top_ten_values

In [ ]:
# save to csv
savename = 'top_ten_hourly_sea_level_by_month_'+ station_id + '.csv'
savepath = data_dir / savename
top_ten_values.to_csv(savename, index=False)


In [ ]:
# resample to monthly max
monthly_max = rsl_hourly.sel(station_id=station_id).resample(time='1MS').max()
mean_monthly_max = monthly_max['sea_level_mhhw'].groupby('time.month').mean()
mean_monthly_max = mean_monthly_max.to_series()

# add average between jan/dec at beginning and end of series for plotting
mean_monthly_max_ext = pd.concat([pd.Series({0: (mean_monthly_max[1]+mean_monthly_max[12])/2}), mean_monthly_max, pd.Series({13: (mean_monthly_max[1]+mean_monthly_max[12])/2})])
mean_monthly_max_ext = mean_monthly_max_ext.sort_index()



## Make a Plot

In [ ]:
# Add month names
df = top_ten_values.copy()
df["month_name"] = df["month"].apply(lambda x: calendar.month_abbr[x])

# Make ranks categorical so Seaborn treats them as discrete (not continuous)
df["rank_str"] = df["rank in month"].astype(int).astype(str)
hue_order = [str(i) for i in range(2, 11)]  # '2'..'10'


fig, ax = plt.subplots(figsize=(8, 6))


ax.scatter(
    x=df[df["rank in month"] == 1]["month_name"],
    y=df[df["rank in month"] == 1]["sea level (m, MHHW)"],
    color="gold",
    edgecolor="black",
    linewidth=0.8,
    label="1",
    s=100
)

# Plot ranks 2–10 in light blue
sns.stripplot(
    data=df[df["rank in month"] > 1],
    x="month_name",
    y="sea level (m, MHHW)",
    hue="rank_str",
    palette="Blues",
    size=8,
    jitter=False,
    alpha=0.9,
    edgecolor="black",
    linewidth=0.5,
    ax=ax
)

# add mean monthly max line
ax.plot(
    mean_monthly_max_ext.index-0.5,
    mean_monthly_max_ext.values,
    color="gray",
    linestyle="--",
    linewidth=2,
    label="Mean Monthly Max"
)

# Annotate rank 1 points with date
for _, row in df[df["rank in month"] == 1].iterrows():
    ax.text(
        row["month_name"],
        row["sea level (m, MHHW)"] + 0.01,  # Slightly above the point
        row["date"].strftime("%Y"),
        fontsize=9,
        ha="center",
        va="bottom",
        rotation=0
    )

# add horizontal line at y=0 for MHHW, annotate
ax.axhline(0, color='black', linestyle=':', linewidth=1)
ax.text(0.5, 0, "MHHW", fontsize=9, ha="center", va="bottom")

# Styling
ax.set_title(f"Top 10 Highest Hourly Sea Level Events by Month\n{df['station_name'].iloc[0]} ({df['station_id'].iloc[0]})", fontsize=14)
ax.set_xlabel("Month", fontsize=14)
ax.set_ylabel("Sea Level (m, MHHW)", fontsize=14)
ax.set_xticks(np.arange(12))
ax.set_xticklabels(rotation=45, fontsize=12, labels=calendar.month_abbr[1:13])
# ax.set_yticklabels(fontsize=12)
ax.legend(bbox_to_anchor=(1.02, 1), loc="upper left", frameon=False, title="Rank in Month", title_fontsize=12)
ax.grid(axis="y", alpha=0.2, linestyle="--")

# set x-limits to show all months
ax.set_xlim(-0.5, 11.5)

# set ylim[1] to be 0.1 m higher than the highest point
ax.set_ylim(bottom=mean_monthly_max_ext.min()-0.1, top=df["sea level (m, MHHW)"].max()+0.1)

# don't cut off the legend, set fig to tight layout
fig.tight_layout()


# save figure
savename = 'SL_Rankings_intra_annual_'+df['station_name'].iloc[0].replace(' ', '_')+'_'+df['station_id'].iloc[0]+'.png'

fig.savefig(output_dir / savename, dpi=300)

## Make a table

In [ ]:
# make a great tables table
from great_tables import GT, md, html
station_name = rsl_hourly['station_name'].sel(station_id=station_id).values

# work on a copy
sub = df[df["rank in month"] <= 5].copy()
sub["Date"] = pd.to_datetime(sub["date"]).dt.strftime("%Y-%m-%d")
sub["Sea Level (cm, MHHW)"] = sub["sea level (m, MHHW)"].round(3)*100

# Pivot into separate tables (rows = Rank, cols = Month num)
dates = sub.pivot(index="rank in month", columns="month", values="Date")
seas  = sub.pivot(index="rank in month", columns="month", values="Sea Level (cm, MHHW)")

# --- stack with keys; we want Sea first, then Date ---
stacked = (
    pd.concat([seas, dates], keys=["Sea Level (cm, MHHW)", "Date"])
      .swaplevel(0, 1)  # rows index now: (rank in month, Field)
)

# Force custom order on the Field level so sorting yields: Sea, Date
field_order = ["Sea Level (cm, MHHW)", "Date"]
lvl0, lvl1 = stacked.index.levels
stacked.index = stacked.index.set_levels(
    [lvl0, pd.CategoricalIndex(lvl1, categories=field_order, ordered=True)],
    level=[0, 1]
)

# Now sort by rank, then Field (respecting our custom order)
stacked = stacked.sort_index(level=[0, 1])

# --- columns Jan..Dec, then prettify to month abbr ---
month_order = list(range(1, 13))
stacked = stacked.reindex(columns=month_order)
stacked.columns = [calendar.month_abbr[m] for m in stacked.columns]

# --- flatten rows (drop the Field label so rows alternate Sea/Date per Rank) ---
table_df = (
    stacked.reset_index()
           .drop(columns="level_1")                 # remove Field column
           .rename(columns={"rank in month": "Rank"})
)

# Take a peak at the data:
table_df.head(12)

Add the mean monthly max on to the table for display:

In [ ]:
# Start with the label
mean_row = {"Rank": "Mean Monthly Max"}

# Add each month abbreviation + its mean
for m in range(1, 13):
    mean_row[calendar.month_abbr[m]] = round(mean_monthly_max.loc[m], 3)*100  # convert to cm

# Convert to DataFrame
mean_row = pd.DataFrame([mean_row])

# Append to bottom (or top) of your table
table_df = pd.concat([table_df, mean_row], ignore_index=True)

In [ ]:
table_df

Now we'll make it pretty:

In [ ]:
from great_tables import GT, style, md, html, loc,vals


date_mask = table_df.apply(lambda row: row.astype(str).str.contains("-").any(), axis=1)
sea_rows = table_df[table_df.iloc[:,1].apply(lambda x: str(x).replace('.','',1).isdigit())].index.tolist()
date_rows = table_df[date_mask].index.tolist()



table_fmt = table_df.copy()
# identify date rows: 0=Sea, 1=Date, 2=Sea, 3=Date, ...
date_rows = table_fmt.index % 2 == 1

# columns to format (all months, not "Rank")
month_cols = [c for c in table_fmt.columns if c != "Rank"]
mean_idx = table_df.index[-1]  # the last row 

def format_yr_mon_day(series):
    # parse to datetime (coerce errors to NaT so non-dates stay as-is)
    dt = pd.to_datetime(series, errors="coerce")
    # format like "2022<br>Jan 6" (no leading zero on day)
    # %-d works on Unix; Windows needs %#d. We'll do a portable strip:
    s_year = dt.dt.strftime("%Y")
    s_mday = dt.dt.strftime("%b %d").str.replace(r"\b0(\d)\b", r"\1", regex=True)
    out = s_year + "<br>" + s_mday
    # keep original where not a valid date (NaT)
    out = np.where(dt.notna(), out, series.astype(str))
    return out

# apply only to the date rows for each month column
for col in month_cols:
    table_fmt.loc[date_rows, col] = format_yr_mon_day(table_fmt.loc[date_rows, col])

table_fmt["Rank"] = table_fmt["Rank"].astype(str)
table_fmt.loc[date_rows, "Rank"] = ""   # or None

min_val = float(table_fmt.iloc[sea_rows, 1:].min().min())
max_val = float(table_fmt.iloc[sea_rows, 1:].max().max())

table_fmt.loc[mean_idx, "Rank"] = "Mean<br>Monthly Max"
# start table
table = GT(table_fmt)


table = (
    table.tab_header(title="Top 5 Sea Levels by Month", subtitle=f"Station: {station_name} ({station_id}), Units: cm, MHHW")
         # smaller text for date rows
         .tab_style(
             style=style.text(size="10px"),
             locations=loc.body(rows=[i for i in table_df.index if i not in sea_rows])
         )
         .tab_style(
             style=style.text(weight="bold"),
             locations=loc.body(rows=mean_idx)
         )
         .cols_align("center", columns=month_cols)
         # color scale applied only to sea-level rows
         .data_color(
             columns=[c for c in table_df.columns if c != "Rank"],  # skip rank
             rows=sea_rows,
             domain=[min_val, max_val],
             palette=["white", "lightblue"],
             na_color="white",
         )
         .fmt_number(columns=month_cols, rows=sea_rows, n_sigfig=3)
         .tab_source_note(source_note="Source: NOAA CO-OPS Hourly Water Level")
)

savename = f"top_5_hourly_sea_level_by_month_{station_id}_table.png"
savepath =  output_dir / savename
print(f"Saving table to {savepath}")
table.save(savepath,scale=2)  # scale=2 for high-DPI displays

# save as raw html too
savename = f"top_5_hourly_sea_level_by_month_{station_id}_table.html"
savepath =  output_dir / savename
table.write_raw_html(savepath)

table

## Make a Map

In [ ]:
# for each station, do the top ten function, but only return the rank 1 event for each month
rank1_events = pd.DataFrame()
for station in rsl_hourly['station_id'].values:
    for month in range(1,13):
        rank1_events = pd.concat([rank1_events, get_top_ten(rsl_hourly, station_id=station, month=month, mode='max').iloc[[0]]])    


Turn it into a dataset for easy plotting:

In [ ]:
# turn rank1_events into an xarray dataset, with coordinates lat, lon, station_id, station_name, month
rank1_events_ds = rank1_events.set_index(['station_id', 'month']).to_xarray()

# drop rank in month, type
rank1_events_ds = rank1_events_ds.drop_vars(['rank in month', 'type'])

# station name var only has station_id as coord
rank1_events_ds['station_name'] = (('station_id'), rank1_events['station_name'].unique())

rank1_events_ds['lat'] = (('station_id'), [rsl_hourly['lat'].sel(station_id=stid).values for stid in rank1_events_ds['station_id'].values])
rank1_events_ds['lon'] = (('station_id'), [rsl_hourly['lon'].sel(station_id=stid).values for stid in rank1_events_ds['station_id'].values])

rank1_events_ds

In [ ]:
# drop Sand Island, Midway Island
rank1_events_ds = rank1_events_ds.drop_sel(station_id=['1619910'])

In [ ]:
# make a 12 panel map, showing the locations of the rank 1 event for each month for each station
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import BoundaryNorm, ListedColormap


# Define discrete boundaries
min_value = rank1_events_ds['sea level (m, MHHW)'].min()
max_value = rank1_events_ds['sea level (m, MHHW)'].max()
min_value, max_value = [0.2, 1]  # set min and max values for color scale
# create boundaries from min to max with interval of 0.1
boundcolors = np.arange(min_value, max_value, 0.1)  # Adjust boundaries as needed
cmap = plt.get_cmap('viridis', len(boundcolors) - 1)  # Create a discrete jet colormap
norm = BoundaryNorm(boundcolors, ncolors=cmap.N, clip=True)



fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(10, 8),   
                         subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)},
                         constrained_layout=True)

months = [calendar.month_name[m] for m in range(1, 13)]


for i, ax in enumerate(axes.flat):
    month = i + 1
    month_data = rank1_events_ds.sel(month=month)
    sc = ax.scatter(
        month_data['lon'],
        month_data['lat'],
        c=month_data['sea level (m, MHHW)'],
        s=150,
        cmap=cmap,
        norm=norm,
        transform=ccrs.PlateCarree(),
        zorder=10
    )
    ax.coastlines(linewidth=0.5)
    ax.add_feature(cfeature.LAND, zorder=0, edgecolor='black', facecolor='lightgray',linewidth=0.5,alpha=0.5)
    # ax.set_title(months[i], location='left', fontsize=14)
    ax.set_extent([198, 206, 18, 23])
    # add gridlines
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, linewidth=0.5, color='gray', alpha=0.8, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False

    # remove gl bottom labels if not in last row
    if i < 9:
        gl.bottom_labels = False
    # remove gl left labels if not in first column
    if i % 3 != 0:
        gl.left_labels = False

# add a colorbar to the right of the figure
cbar = fig.colorbar(sc, ax=axes, orientation='vertical', fraction=0.02, pad=0.04,extend='both')
cbar.set_label('Sea Level (m, MHHW)', fontsize=12)
# cbar.ax.tick_params(labelsize=10)        

# set font to Avenir
plt.rcParams["font.family"] = "Avenir, Helvetica, Arial, sans-serif"

# add title to the figure
fig.suptitle('Highest Hourly Sea Level Event by Month at NOAA Hawaii Stations', fontsize=16)

# add month labels to each subplot
for i, ax in enumerate(axes.flat):
    month = i + 1
    ax.text(0.8, 0.8, months[i], transform=ax.transAxes, ha='center', va='bottom', fontsize=14)

# save figure
savename = 'SL_Rankings_intra_annual_map.png'
fig.savefig(output_dir / savename, dpi=300)